# Differential equationas - Taylor approximations

In [240]:
from IPython.display import display, Markdown
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))

In [241]:
%run -i 'implicit.py'

We encode differential equations just as in the attempt using Fourier series, but now implement taylor approximations

In [242]:
class PDE():
    def __init__(self, k , phi, phi_var):
        if len(phi_var) != k+2:
            raise(Exception('wrong number of functions given, should be a list {} long'.format(k)))
        self.k = k # order of the PDE
        self.phi = phi # list of $k + 1$ functions
        self.phi_var = phi_var # should be a dictionary
        
    
    def __str__(self):
        # reformat phi so shows as a pde
        out = latex(self.phi)
        for var in self.phi_var.keys():
            if var == 'x':
                continue
            out = out.replace(latex(self.phi_var[var]), var)
            
        return out
    
    def _latex_(self):
        return str(self)               
            
        
        
    def taylor_approx_raw(self, x, an, n = 2, interval = (0,1)):
        # n is the order of the Fourier approximation
        # x should be a SageMath variable
        if len(an) != n + 1:
            raise(Exception('Should be {} coefficients'.format(n + 1)))
        
        # first generate the taylor approx f(x)
        
        f(x) = an[0] 
        for i in range(1,n+1):
            f(x) = f(x) + (an[i]*x^i)/(factorial(i))

        temp(x) = x
        fr_vars = [temp, f]
        if self.k > 0:
            for i in range(1, self.k +1):
                f = f.diff()[0] # returns a matrix of one, so take first elt
                fr_vars.append(f)
        out(x) = phi(*fr_vars)
        
        return out
    
    def taylor_approx_l2(self, x, an, n = 2, interval = (0,1)):
        raw = self.taylor_approx_raw(x, an, n, interval)
        out = [0]*(n+1)
        
        # first get a0
        a0 = integral(raw, (x,interval[0],interval[1]))
        
        out[0] = a0
        for i in range(1, n+1):
            ai = integral(raw * x^i, (x,interval[0],interval[1]))
            out[i] = ai/(factorial(i))

        return out
    
    
    def taylor_approx_at0(self, x, an, n = 2):
        raw = self.taylor_approx_raw(x, an, n, interval)
        out = [0]*(n+1)
        
        for i in range(0, n+1):
            ai = raw(x = 0)
            raw = raw.diff(x)
            out[i] = ai/factorial(i)
        return out
        
    def taylor_approx_atx(self, x, an, n = 2):
        raw = self.taylor_approx_raw(x, an, n, interval)
        out = [0]*(n+1)
        
        for i in range(0, n+1):
            ai = raw(x)
            raw = raw.diff(x)
            out[i] = ai/factorial(i)
        return out
        
    # TODO overide display method, so can view the PDE

In [243]:
var('x a0 a1 a2 a3 a4 a5 a6 a7 x0 x1 x2')
var('l', latex_name=r'\lambda')

l

In [244]:
phi(x, x0, x1, x2) = x2 + l * sin(x0) 
lprint(phi)

$ \left( x, x_{0}, x_{1}, x_{2} \right) \ {\mapsto} \ {\lambda} \sin\left(x_{0}\right) + x_{2} $

In [245]:
pde = PDE(2, phi, phi_var = {'x' : x, 'f(x)' : x0, 'f^{1}(x)' : x1, 'f^{2}(x)' : x2} )

In [246]:
lprint(pde)

$ \left( x, f(x), f^{1}(x), f^{2}(x) \right) \ {\mapsto} \ {\lambda} \sin\left(f(x)\right) + f^{2}(x) $

In [247]:
f = pde.taylor_approx_raw(x, [a0, a1, a2, a3, a4, a5], n = 5)
lprint(f)

$ x \ {\mapsto}\ \frac{1}{6} \, a_{5} x^{3} + \frac{1}{2} \, a_{4} x^{2} + a_{3} x + {\lambda} \sin\left(\frac{1}{120} \, a_{5} x^{5} + \frac{1}{24} \, a_{4} x^{4} + \frac{1}{6} \, a_{3} x^{3} + \frac{1}{2} \, a_{2} x^{2} + a_{1} x + a_{0}\right) + a_{2} $

In [248]:
f(x = 0)

l*sin(a0) + a2

In [249]:
flist = pde.taylor_approx_at0(x, [a0,a1,a2,a3,a4,a5], n = 5)

In [250]:
lprint(flist)

$ \left[{\lambda} \sin\left(a_{0}\right) + a_{2}, a_{1} {\lambda} \cos\left(a_{0}\right) + a_{3}, -\frac{1}{2} \, a_{1}^{2} {\lambda} \sin\left(a_{0}\right) + \frac{1}{2} \, a_{2} {\lambda} \cos\left(a_{0}\right) + \frac{1}{2} \, a_{4}, -\frac{1}{6} \, a_{1}^{3} {\lambda} \cos\left(a_{0}\right) - \frac{1}{2} \, a_{1} a_{2} {\lambda} \sin\left(a_{0}\right) + \frac{1}{6} \, a_{3} {\lambda} \cos\left(a_{0}\right) + \frac{1}{6} \, a_{5}, \frac{1}{24} \, a_{1}^{4} {\lambda} \sin\left(a_{0}\right) - \frac{1}{4} \, a_{1}^{2} a_{2} {\lambda} \cos\left(a_{0}\right) - \frac{1}{8} \, a_{2}^{2} {\lambda} \sin\left(a_{0}\right) - \frac{1}{6} \, a_{1} a_{3} {\lambda} \sin\left(a_{0}\right) + \frac{1}{24} \, a_{4} {\lambda} \cos\left(a_{0}\right), \frac{1}{120} \, a_{1}^{5} {\lambda} \cos\left(a_{0}\right) + \frac{1}{12} \, a_{1}^{3} a_{2} {\lambda} \sin\left(a_{0}\right) - \frac{1}{8} \, a_{1} a_{2}^{2} {\lambda} \cos\left(a_{0}\right) - \frac{1}{12} \, a_{1}^{2} a_{3} {\lambda} \cos\left(a_{0}\right) - \frac{1}{12} \, a_{2} a_{3} {\lambda} \sin\left(a_{0}\right) - \frac{1}{24} \, a_{1} a_{4} {\lambda} \sin\left(a_{0}\right) + \frac{1}{120} \, a_{5} {\lambda} \cos\left(a_{0}\right)\right] $

## Applying implicit function theorem

In [251]:
f1(a0, a1, a2, a3, a4, a5, l) = flist[0]
f2(a0, a1, a2, a3, a4, a5, l) = flist[1]
f3(a0, a1, a2, a3, a4, a5, l) = flist[2]
f4(a0, a1, a2, a3, a4, a5, l) = flist[3]
f5(a0, a1, a2, a3, a4, a5, l) = flist[4]
f6(a0, a1, a2, a3, a4, a5, l) = flist[5]


funcs = [f1,f2,f3,f4,f5,f6]

In [252]:
[func(0,0,0,0,0,0,l) for func in funcs]

[0, 0, 0, 0, 0, 0]

In [253]:
J = jacobian(funcs, (a0, a1, a2, a3, a4, a5))(a0=0, a1=0, a2=0, a3 = 0, a4 = 0, a5 = 0, l=0 )
lprint(J)

$ \left(\begin{array}{rrrrrr}
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & \frac{1}{2} & 0 \\
0 & 0 & 0 & 0 & 0 & \frac{1}{6} \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0
\end{array}\right) $

The matrix has full rank

In [254]:
K = J.right_kernel().basis()
lprint(K)

$ \left[\left(1,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,1,\,0,\,0,\,0,\,0\right)\right] $

In [255]:
I = J.image().basis()
lprint(I)

$ \left[\left(0,\,0,\,1,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,1,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,1,\,0\right), \left(0,\,0,\,0,\,0,\,0,\,1\right)\right] $

In [256]:
lprint(f6.diff(a3))

$ \left( a_{0}, a_{1}, a_{2}, a_{3}, a_{4}, a_{5}, {\lambda} \right) \ {\mapsto} \ -\frac{1}{12} \, a_{1}^{2} {\lambda} \cos\left(a_{0}\right) - \frac{1}{12} \, a_{2} {\lambda} \sin\left(a_{0}\right) $

In [257]:
funcs = [f1, f2, f3, f4] # cokernel directions
position = {'a0' : 0, 'a1' : 0, 'a2' : 0, 'a3' : 0, 'a4' : 0, 'a5' : 0, 'l' : 0}
var_dict = {'y1' : a2, 'y2' : a3, 'y3' : a4, 'y4' : a5, 'x1' : a0, 'x2' : a1, 'x3' : l}
x_var_keys = {'x1', 'x2', 'x3'}
x_dim = 3
y_dim = 4

In [258]:
t_dict = TensorDict(funcs, position, var_dict, x_dim, y_dim)

computing Taylor approximaton to 3 order for speedup


In [259]:
out = get_hkx_polynomial(funcs, 2, x_dim, y_dim, var_dict, x_var_keys, t_dict, position)

100%|██████████| 4/4 [00:03<00:00,  1.33it/s]


In [260]:
lprint(out)

$ \left[-a_{0} {\lambda}, -a_{1} {\lambda}, 0, 0\right] $

In [32]:
t_dict['fY'].tensors[0].data

,data
1,
y1,sin(1)
y2,cos(1) + sin(1) - 1
y3,cos(1) - 1/2*sin(1)


In [28]:
lprint(f3)

$ \left( a_{0}, a_{1}, a_{2}, {\lambda} \right) \ {\mapsto} \ -\frac{1}{6} \, a_{0}^{2} - \frac{1}{4} \, {\left(a_{0} + 6 \, \cos\left(1\right) + 10 \, \sin\left(1\right)\right)} a_{1} - \frac{1}{10} \, a_{1}^{2} - \frac{1}{60} \, {\left(6 \, a_{0} + 5 \, a_{1} + 300 \, \cos\left(1\right) - 195 \, \sin\left(1\right)\right)} a_{2} - \frac{1}{56} \, a_{2}^{2} + \frac{1}{24} \, {\left(4 \, a_{1} + 3 \, a_{2}\right)} {\lambda} + \frac{1}{2} \, a_{0} {\left(2 \, \cos\left(1\right) - \sin\left(1\right)\right)} + 3 \, a_{1} $